In [27]:
pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install sklearn

  Using cached sklearn-0.0.post10.tar.gz (3.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post10-py3-none-any.whl size=2959 sha256=a9faf92c320a5fac981c3722214dd711f4d0131517aeafcc10ada18f9862bd19
  Stored in directory: /Users/zhiqiaogong/Library/Caches/pip/wheels/40/ff/aa/d5ea39ea054381b6eccc737df85530f8637e17f02517284f3f
Successfully built sklearn

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [385]:
import random
import sklearn
from sklearn import linear_model
from matplotlib import pyplot as plt
from collections import defaultdict
import gzip
import math

In [386]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [387]:
answers = {}

In [388]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

In [389]:
data = list(parseData("beer_50000.json"))

In [390]:
random.seed(0)
random.shuffle(data)

In [391]:
dataTrain = data[:25000]
dataValid = data[25000:37500]
dataTest = data[37500:]

In [392]:
maxLen  = max([len(d['review/text']) for d in dataTrain])

In [393]:
yTrain = [d['beer/ABV'] > 7 for d in dataTrain]
yValid = [d['beer/ABV'] > 7 for d in dataValid]
yTest = [d['beer/ABV'] > 7 for d in dataTest]

In [394]:
dataTrain[0]

{'review/appearance': 4.0,
 'beer/style': 'Belgian Pale Ale',
 'review/palate': 4.0,
 'review/taste': 4.0,
 'beer/name': 'La Binchoise Blonde Tradition',
 'review/timeUnix': 1210043435,
 'beer/ABV': 6.5,
 'beer/beerId': '7693',
 'beer/brewerId': '3282',
 'review/timeStruct': {'isdst': 0,
  'mday': 6,
  'hour': 3,
  'min': 10,
  'sec': 35,
  'mon': 5,
  'year': 2008,
  'yday': 127,
  'wday': 1},
 'review/overall': 4.0,
 'review/text': 'From the 11.2oz stubby.\tPours a straw/golden color with a nice head that soon settles to a nice thin crown. Pleasing aroma of yeast and fruit. Taste melds fruit notes with a nice carbonation and peppery hop profile. Slightly dry finish, but satisfying leaving you wanting another drink. Recommended.',
 'user/profileName': 'sinistermadman',
 'review/aroma': 4.0}

In [395]:
categoryCounts = defaultdict(int)
for d in data:
    categoryCounts[d['beer/style']] += 1

In [396]:
categories = [c for c in categoryCounts if categoryCounts[c] > 1000]

In [397]:
catID = dict(zip(list(categories),range(len(categories))))

In [398]:
def myonehotcat(i):
    t = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    if i:
        t[i] = 1
    return t 

In [399]:
def myFeat(d, incat, inrev, inlen):
    if incat and not inrev and not inlen:
        t = [0,0,0,0,0,0,0,0,0,0,0,0,0]
        i = catID.get(d['beer/style'])
        if i:
            t[i] = 1

    elif not incat and inrev and inlen:
        t = [0,0,0,0,0,  0]
        t[0] = d['review/appearance']
        t[1] = d['review/palate']
        t[2] = d['review/taste']
        t[3] = d['review/overall']
        t[4] = d['review/aroma']
        t[5] = len(d['review/text'])/maxLen

    elif incat and not inrev and inlen:
        t = [0,0,0,0,0,0,0,0,0,0,0,0,0,  0]

        i = catID.get(d['beer/style'])
        if i:
            t[i] = 1

        t[13] = len(d['review/text'])/maxLen

    elif incat and inrev and not inlen:
        t = [0,0,0,0,0,0,0,0,0,0,0,0,0,  0,0,0,0,0]

        i = catID.get(d['beer/style'])
        if i:
            t[i] = 1

        t[13] = d['review/appearance']
        t[14] = d['review/palate']
        t[15] = d['review/taste']
        t[16] = d['review/overall']
        t[17] = d['review/aroma']

    elif incat and inrev and inlen:
        t = [0,0,0,0,0,0,0,0,0,0,0,0,0,  0,0,0,0,0,  0]

        i = catID.get(d['beer/style'])
        if i:
            t[i] = 1

        t[13] = d['review/appearance']
        t[14] = d['review/palate']
        t[15] = d['review/taste']
        t[16] = d['review/overall']
        t[17] = d['review/aroma']

        t[18] = len(d['review/text'])/maxLen
        
    return t 

In [400]:
def myber(pred, y):
    TP = sum([(p and l) for (p,l) in zip(pred, y)])
    TN = sum([(not p and not l) for (p,l) in zip(pred, y)])
    FP = sum([(p and not l) for (p,l) in zip(pred, y)])
    FN = sum([(not p and l) for (p,l) in zip(pred, y)])

    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    BER = 1 - 1/2 * (TPR + TNR)

    return BER

In [401]:
def feat(includeCat = True, includeReview = True, includeLength = True):

    featX =  [myFeat(d, includeCat, includeReview, includeLength) for d in dataTrain]
    featXV = [myFeat(d, includeCat, includeReview, includeLength) for d in dataValid]
    featXT = [myFeat(d, includeCat, includeReview, includeLength) for d in dataTest]

    
    return featX, featXV, featXT

In [402]:
def pipeline(reg, includeCat = True, includeReview = True, includeLength = True):
    featX, featXV, featXT = feat(includeCat, includeReview, includeLength)

    mod = sklearn.linear_model.LogisticRegression(C = reg, class_weight='balanced')
    mod.fit(featX,yTrain)
    predictionsV = mod.predict(featXV)
    predictionsT = mod.predict(featXT)

    return mod, myber(predictionsV, yValid), myber(predictionsT, yTest)

In [403]:
### Question 1

In [404]:
mod, validBER, testBER = pipeline(10, True, False, False)
validBER, testBER

(0.16130237168160533, 0.1607838024608832)

In [405]:
answers['Q1'] = [validBER, testBER]

In [406]:
assertFloatList(answers['Q1'], 2)

In [407]:
### Question 2

In [408]:
mod, validBER, testBER = pipeline(10, True, True, True)
validBER, testBER

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.14966582360742176, 0.151303548307838)

In [409]:
answers['Q2'] = [validBER, testBER]

In [410]:
assertFloatList(answers['Q2'], 2)

In [411]:
### Question 3

In [412]:
dataTrain

[{'review/appearance': 4.0,
  'beer/style': 'Belgian Pale Ale',
  'review/palate': 4.0,
  'review/taste': 4.0,
  'beer/name': 'La Binchoise Blonde Tradition',
  'review/timeUnix': 1210043435,
  'beer/ABV': 6.5,
  'beer/beerId': '7693',
  'beer/brewerId': '3282',
  'review/timeStruct': {'isdst': 0,
   'mday': 6,
   'hour': 3,
   'min': 10,
   'sec': 35,
   'mon': 5,
   'year': 2008,
   'yday': 127,
   'wday': 1},
  'review/overall': 4.0,
  'review/text': 'From the 11.2oz stubby.\tPours a straw/golden color with a nice head that soon settles to a nice thin crown. Pleasing aroma of yeast and fruit. Taste melds fruit notes with a nice carbonation and peppery hop profile. Slightly dry finish, but satisfying leaving you wanting another drink. Recommended.',
  'user/profileName': 'sinistermadman',
  'review/aroma': 4.0},
 {'review/appearance': 4.0,
  'beer/style': 'American Amber / Red Ale',
  'review/palate': 3.0,
  'review/taste': 3.0,
  'beer/name': 'Kirkland Signature Amber Ale',
  'revie

In [413]:
minvber = 1
bestc = 0
for c in [0.001, 0.01, 0.1, 1, 10]:
    mod, validBER, testBER = pipeline(c, True, True, True)
    if validBER < minvber:
        minvber = validBER
        bestc = c
    print(str(c) +":"+  str(validBER) +" "+ str(testBER))
bestc

0.001:0.19678157220174275 0.19991857226028675
0.01:0.1508904905448305 0.1510223131273556


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.1:0.151781711619823 0.1510990710004918


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1:0.14978805879512525 0.15088356940801062
10:0.14966582360742176 0.151303548307838


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


10

In [414]:
bestC = bestc

In [415]:
mod, validBER, testBER = pipeline(bestC, True, True, True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [416]:
answers['Q3'] = [bestC, validBER, testBER]

In [417]:
assertFloatList(answers['Q3'], 3)

In [418]:
### Question 4

In [419]:
mod, validBER, testBER_noCat = pipeline(1, False, True, True)

In [420]:
mod, validBER, testBER_noReview = pipeline(1, True, False, True)

In [421]:
mod, validBER, testBER_noLength = pipeline(1, True, True, False)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [425]:
answers['Q4'] = [testBER_noCat, testBER_noReview, testBER_noLength]
testBER_noCat, testBER_noReview, testBER_noLength

(0.3139492057092712, 0.16086665202428674, 0.15446717041296165)

In [426]:
assertFloatList(answers['Q4'], 3)

In [424]:
### Question 5

In [428]:
path = "amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
f = gzip.open(path, 'rt', encoding="utf8")

header = f.readline()
header = header.strip().split('\t')

In [429]:
header

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [430]:
dataset = []

pairsSeen = set()

for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    ui = (d['customer_id'], d['product_id'])
    if ui in pairsSeen:
        print("Skipping duplicate user/item:", ui)
        continue
    pairsSeen.add(ui)
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

Skipping duplicate user/item: ('46953315', 'B00QM3CNN6')
Skipping duplicate user/item: ('31616428', 'B0026RB0G8')
Skipping duplicate user/item: ('47240912', 'B008I653SC')
Skipping duplicate user/item: ('14503091', 'B003FRMRC4')
Skipping duplicate user/item: ('38538360', 'B00HVLUR86')
Skipping duplicate user/item: ('43448024', 'B00HVLUR86')
Skipping duplicate user/item: ('51525270', 'B00HVLUR86')
Skipping duplicate user/item: ('20652160', 'B004OU2IQG')
Skipping duplicate user/item: ('10964440', 'B00HVLUR86')
Skipping duplicate user/item: ('20043677', 'B00HVLUR86')
Skipping duplicate user/item: ('44796499', 'B00HVLUSGM')
Skipping duplicate user/item: ('29066899', 'B0002CZSYO')
Skipping duplicate user/item: ('10385056', 'B004OU2IQG')
Skipping duplicate user/item: ('1658551', 'B00HVLURL8')
Skipping duplicate user/item: ('907433', 'B00N9Q2E5G')
Skipping duplicate user/item: ('39412969', 'B00HVLUR86')
Skipping duplicate user/item: ('4901688', 'B00HVLUR86')
Skipping duplicate user/item: ('234

In [431]:
dataTrain = dataset[:int(len(dataset)*0.9)]
dataTest = dataset[int(len(dataset)*0.9):]
dataTrain

[{'marketplace': 'US',
  'customer_id': '45610553',
  'review_id': 'RMDCHWD0Y5OZ9',
  'product_id': 'B00HH62VB6',
  'product_parent': '618218723',
  'product_title': 'AGPtek® 10 Isolated Output 9V 12V 18V Guitar Pedal Board Power Supply Effect Pedals with Isolated Short Cricuit / Overcurrent Protection',
  'product_category': 'Musical Instruments',
  'star_rating': 3,
  'helpful_votes': 0,
  'total_votes': 1,
  'vine': 'N',
  'verified_purchase': 'N',
  'review_headline': 'Three Stars',
  'review_body': 'Works very good, but induces ALOT of noise.',
  'review_date': '2015-08-31'},
 {'marketplace': 'US',
  'customer_id': '14640079',
  'review_id': 'RZSL0BALIYUNU',
  'product_id': 'B003LRN53I',
  'product_parent': '986692292',
  'product_title': 'Sennheiser HD203 Closed-Back DJ Headphones',
  'product_category': 'Musical Instruments',
  'star_rating': 5,
  'helpful_votes': 0,
  'total_votes': 0,
  'vine': 'N',
  'verified_purchase': 'Y',
  'review_headline': 'Five Stars',
  'review_body'

In [432]:
# Feel free to keep or discard

usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair
reviewsPerUser = defaultdict(list)

for d in dataTrain:
    user,item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['star_rating']
    itemNames[item] = d['product_title']


In [433]:
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

ratingMean = sum([d['star_rating'] for d in dataset]) / len(dataset)

In [434]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [435]:
def mostSimilar(i, N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:N]

In [436]:
query = 'B00KCHRKD6'

In [437]:
ms = mostSimilar(query, 10)
ms

[(0.015228426395939087, 'B00H7NFDKA'),
 (0.014492753623188406, 'B00QKVV3HC'),
 (0.014492753623188406, 'B00GXRMD7W'),
 (0.014084507042253521, 'B00H7ILRRI'),
 (0.014084507042253521, 'B0057RUMPO'),
 (0.014084507042253521, 'B000B6DTYW'),
 (0.013888888888888888, 'B00L2708TI'),
 (0.013513513513513514, 'B009Z1KKWI'),
 (0.013513513513513514, 'B000VYINCW'),
 (0.013333333333333334, 'B003F2BDZQ')]

In [438]:
answers['Q5'] = ms

In [439]:
assertFloatList([m[0] for m in ms], 10)

In [440]:
### Question 6

In [441]:
def MSE(y, ypred):
    return sum([(p - l)**2 for (p,l) in zip(y, ypred)])/len(y)

In [442]:
def predictRating(user,item):
    numerator = 0
    denominator = 0
    
    # If the item hasn't been seen before, return the global average rating
    if item not in itemAverages:
        return ratingMean

    # Calculate predicted rating based on the formula
    for j in itemsPerUser[user]:  # Iterating over items that the user has rated
        if j == item:
            continue
        similarity = Jaccard(usersPerItem[item], usersPerItem[j])
        numerator += (ratingDict[(user, j)] - itemAverages[j]) * similarity
        denominator += similarity

    # If no similar items exist (denominator is zero), return the item's average rating
    if denominator == 0:
        return itemAverages[item]

    return itemAverages[item] + numerator / denominator

In [443]:
simPredictions = [predictRating(d['customer_id'], d['product_id']) for d in dataTest] 

In [444]:
labels = [d['star_rating'] for d in dataTest]

In [445]:
answers['Q6'] = MSE(simPredictions, labels)
answers['Q6']

1.7151690273245603

In [446]:
assertFloat(answers['Q6'])

In [447]:
### Question 7

In [448]:
from datetime import datetime

usersPerItemT = defaultdict(set) # Maps an item to the users who rated it
itemsPerUserT = defaultdict(set) # Maps a user to the items that they rated
date_format = "%Y-%m-%d"
ratingDictTime = {}

for d in dataTrain:
    user,item = d['customer_id'], d['product_id']
    usersPerItemT[item].add(user)
    itemsPerUserT[user].add(item)

    timestamp_dt = datetime.strptime(d['review_date'], date_format)
    
    ratingDictTime[(user, item)] = {
        'rating': d['star_rating'],
        'timestamp': timestamp_dt
    }

In [449]:
userAveragesT = {}
itemAveragesT = {}

for u in itemsPerUserT:
    rs = [ratingDictTime[(u,i)]['rating'] for i in itemsPerUserT[u]]
    userAveragesT[u] = sum(rs) / len(rs)
    
for i in usersPerItemT:
    rs = [ratingDictTime[(u,i)]['rating'] for u in usersPerItemT[i]]
    itemAveragesT[i] = sum(rs) / len(rs)

ratingMeanT = sum([d['star_rating'] for d in dataset]) / len(dataset)

In [450]:
def myPridictRating(user, item, mylambda):
    numerator = 0
    denominator = 0

    if item not in itemAveragesT:
        return ratingMeanT

    if user not in usersPerItemT:
        return ratingMeanT
    
    # Calculate predicted rating based on the formula
    for j in itemsPerUser[user]:  # Iterating over items that the user has rated
        if j == item:
            continue
            
        similarity = Jaccard(usersPerItemT[item], usersPerItemT[j])
        
        # Calculate time difference in days
        time_difference = abs((ratingDictTime[(user, item)]['timestamp'] - ratingDictTime[(user, j)]['timestamp']).days)
        decay_factor = math.exp(-mylambda * time_difference)
           
        numerator += (ratingDictTime[(user, j)]['rating'] - userAveragesT[user]) * sim_ij * decay_factor
        denominator += abs(sim_ij * decay_factor)
        
    if denominator == 0:
        if item in itemAveragesT:
            return itemAveragesT[item]
        return ratingMeanT
        
    return userAveragesT[user] + numerator / denominator


In [451]:
pd = [myPridictRating(d['customer_id'], d['product_id'], 10) for d in dataTest]

In [452]:
bestmse = MSE(pd, labels)
bestmse

1.6200494120785502

In [453]:
itsMSE = bestmse

In [454]:
answers['Q7'] = ["Convert the time string to a datetime object and calculate the difference in days between the two as f(tu,j)", itsMSE]

In [455]:
assertFloat(answers['Q7'][1])

In [456]:
f = open("answers_hw2.txt", 'w')
f.write(str(answers) + '\n')
f.close()

In [ ]:
pip install jupyter_contrib_nbextensions

SyntaxError: invalid syntax (2123509818.py, line 1)